# Lab 1 — Data Lake Ingest & Query (AWS ML Associate)

This notebook guides you through:
1) Generate sample CSV + Parquet data
2) Upload to S3
3) Create Glue Database + Crawler
4) Query with Athena

⚠️ **Prereqs**
- AWS CLI configured (`aws configure`)
- IAM permissions for S3, Glue, Athena
- Define an S3 bucket you own in `LAB_BUCKET`


In [ ]:
# --- 0) Configure env vars ---
import os
LAB_BUCKET = 's3://CHANGE-ME-BUCKET'  # TODO: replace with your bucket
LAB_PREFIX = 'ml-assoc/l1'
print('LAB_BUCKET =', LAB_BUCKET)
print('LAB_PREFIX =', LAB_PREFIX)


## 1) Generate Sample Data (CSV + Parquet)

In [ ]:
import pandas as pd, numpy as np, os
n = 1000
df = pd.DataFrame({
  'customer_id': np.arange(n),
  'age': np.random.randint(18, 80, n),
  'income': np.random.normal(70000, 15000, n).round(2),
  'state': np.random.choice(list('ABCDE'), n),
  'churn': np.random.choice([0,1], n, p=[0.8,0.2])
})
os.makedirs('/tmp/l1', exist_ok=True)
df.to_csv('/tmp/l1/customers.csv', index=False)
df.to_parquet('/tmp/l1/customers.parquet', index=False)
df.head()

## 2) Upload to S3

In [ ]:
!aws s3 cp /tmp/l1/ {LAB_BUCKET}/{LAB_PREFIX}/data/ --recursive

## 3) Glue: Create Database + Crawler

In [ ]:
import random, time, subprocess
CRAWLER = f'l1-crawler-{random.randint(1000,9999)}'
print('Crawler name:', CRAWLER)
!aws glue create-database --database-input Name=l1_db || true
!aws glue create-crawler --name {CRAWLER} --role AWSGlueServiceRoleDefault --database-name l1_db --targets S3Targets=[{{Path='{LAB_BUCKET}/{LAB_PREFIX}/data/'}}]
!aws glue start-crawler --name {CRAWLER}

## 4) Athena Query

In [ ]:
!aws s3 mb {LAB_BUCKET}/{LAB_PREFIX}/query/ || true
!aws athena start-query-execution --query-string "SELECT state, AVG(income) avg_income, AVG(churn) churn_rate FROM l1_db.\"customers_parquet\" GROUP BY state;" --work-group primary --result-configuration OutputLocation={LAB_BUCKET}/{LAB_PREFIX}/query/

## 5) Teardown (optional)

In [ ]:
!aws glue delete-crawler --name {CRAWLER}
!aws s3 rm {LAB_BUCKET}/{LAB_PREFIX} --recursive